# TME 4 - KOROGLU Mathis et FAURE Guillaume

$n_e$ : nombre d'équipes participantes  
$n_s$ : nombre de semaines de matchs  
$n_j$ : nombre jour de matchs = $2*n_s$  
$m_{j,x,y}$ codé par $v_k = j*n_e²+x*n_e+y+1$  


# EXERCICE 1

In [29]:
import os
import time
import subprocess

# Question 1

Le nombre de variables propositionnelles $m_{j,x,y}$ est de $n_e*n_j²$

# Question 2

In [1]:
def codage(ne,nj,j,x,y) :
    return j*(ne**2)+x*ne+y+1

# Question 3

In [2]:
def decodage(k,ne):  
    j = (k-1) // (ne**2)
    x = ((k-1)-(j*(ne**2))) // ne
    y = (k-1) % ne
    return(j,x,y)

In [3]:
import random
ne= random.randint(2,100)
nj= random.randint(2,100)
j= random.randint(0,nj)
x= random.randint(0,ne)
y= random.randint(0,ne)
while x == y:
    y= random.randint(0,ne)
k=codage(ne,nj,j,x,y)
print(f'Data : ne={ne}, nj={nj}, j={j}, x={x}, y={y}')
print(f'codage : k={k}')
j,x,y =decodage(k,ne)
print(f'decodage : j={j}, x={x}, y={y}')

Data : ne=20, nj=92, j=62, x=16, y=19
codage : k=25140
decodage : j=62, x=16, y=19


# EXERCICE 3

# Question 1

1. ”au moins une de ces variables est vraie” 

J$\in${0,...,n<sub>j-1</sub>} et E$\in${0,...,n<sub>e-1</sub>}

Formule et encodage de "au moins une de ces variables est vraie" :  
$$\sum_{i\in E}v_i\ge 1\equiv\bigvee_{i\in E}v_i$$  

Cela correspond donc à une seule clause comportant une disjonction de n littéraux.

In [4]:
def au_moins(liste):
    f_dimacs = ""
    for i in liste:
        f_dimacs+= f'{i} '
    f_dimacs+='0\n'
    return f_dimacs

In [5]:
ex = [-1,2,-3,-4,5,6,7,-8]
res = au_moins(ex)
print(res)

-1 2 -3 -4 5 6 7 -8 0



2. ”au plus une de ces variables est vraie” 

Formule et encodage de "au plus une de ces variables est vraie" :  
$$\sum_{i\in E}v_i\le 1 \equiv \bigwedge_{i\leq j\le n} ^{i\le j\leq n} \neg v_i \vee \neg v_j$$

Cela correspond à $\frac{n*(n-1)}{2}$ clauses de deux littéraux :  O(n²)

In [6]:
def au_plus(liste):
    f_dimacs = ""
    for i in range(len(liste)):
        for j in range(i+1,len(liste)):
            f_dimacs+= f'{-liste[i]} {-liste[j]} 0\n'
    return f_dimacs

In [7]:
ex = [-1,2,-3,-4,5]
res = au_plus(ex)
print(f'nombre de clauses attendue : {int((len(ex)*(len(ex)-1)/2))}')
print(res)

nombre de clauses attendue : 10
1 -2 0
1 3 0
1 4 0
1 -5 0
-2 3 0
-2 4 0
-2 -5 0
3 4 0
3 -5 0
4 -5 0



# Question 2

1. Chaque équipe ne peut jouer plus d'un match par jour:

$$\forall j\in J,\forall e\in E,\sum_{i\in E}(m_{j,i,e}+m_{j,e,i})\le1$$  

2. fonction encoderC1

In [8]:
def encoderC1(ne,nj):
    f_dimacs =""
    liste_e=[e for e in range(ne)]
    for j in range(nj):
        for e in range(ne):
            f_dimacs += au_plus([codage(ne,nj,j,i,e) for i in liste_e] + [codage(ne,nj,j,e,i) for i in liste_e])
    return f_dimacs

3. calcul nombre de contraintes de cardinalité et nombre de clauses

Soit $n_e=3$ et $n_j=4$ donc $n_e*n_j=12$ contraintes de cardinalités (chaque ´equipe ne peut jouer plus d’un match par jour).<br>
On a vu plus haut qu'un encodage par paire 'au plus' nécessitait $\frac{n*(n-1)}{2}$ clauses.<br>
Sachant qu'ici n=$2n_e$, on a $\frac{2n_e*(2n_e-1)}{2}= 15$ clauses par contrainte.

Au total $n_e*n_j*\frac{2n_e*(2n_e-1)}2=180$ clauses pour la contrainte C1.

In [9]:
ne=3
nj=4
contrainte = encoderC1(3,4)
theorie = ne * nj * 2 * ne * (2 * ne - 1) // 2
pratique = len(contrainte.split('0\n'))-1

print(f'nombre de clauses en théorie: {int(theorie)}\nnombre de clauses en pratique: {int(pratique)}')

nombre de clauses en théorie: 180
nombre de clauses en pratique: 180


4. ”Sur la durée du championnat, chaque équipe doit rencontrer l’ensemble des autres équipes une fois à domicile et une fois à l’extérieur, soit exactement 2 matchs par équipe adverse.”

$$\forall (x,y)\in E², x\ne y,\sum_{j\in J}m_{j,x,y}=1$$

5. fonction encoderC2

In [10]:
def encoderC2(ne,nj):
    f_dimacs=''
    liste_j = [j for j in range(nj)]
    for x in range(ne):
        for y in range(ne):
            if x != y:
                f_dimacs+= au_moins([codage(ne,nj,j,x,y) for j in liste_j])
                f_dimacs+= au_plus([codage(ne,nj,j,x,y) for j in liste_j])
    return f_dimacs

6. calcul nombre de contraintes de cardinalité et nombre de clauses

Pour chaque couple d'équipe distincte, elle s'affronte exactement deux fois, une fois à domicile, une fois à l'exterieur cela signifie qu'il y a : $n_e*(n_e-1)=6$ contraintes de cardinalité.<br>

Pour chacune contrainte, il y a pour au_plus : $n_j*(n_j-1)/2=6$ clauses et pour au_moins: $1$ clause<br>

Il y donc au total $n_e*(n_e-1)\left(\frac{n_j*(n_j-1)}2+1\right)=42$ clauses pour C2

In [11]:
ne=3
nj=4
contrainte = encoderC2(3,4)
theorie = ne * (ne-1) * ((nj * (nj - 1)// 2)+1)
pratique = len(contrainte.split('0\n'))-1

print(f'nombre de clauses en théorie: {int(theorie)}\nnombre de clauses en pratique: {int(pratique)}')

nombre de clauses en théorie: 42
nombre de clauses en pratique: 42


7. Ecrire un programme encoder qui encode toutes les contraintes C1 et C2 pour ne et nj donné.

In [12]:
def encoder(ne,nj,nom):
    f_dimacs=''
    f_dimacs+=encoderC1(ne,nj)
    f_dimacs+=encoderC2(ne,nj)
    nb_clauses = len(f_dimacs.split("0\n"))-1
    nb_variables = nj*(ne**2)
    fichier_cnf =f'p cnf {nb_variables} {nb_clauses}\n' + f_dimacs
    with open(nom, "w") as fichier:     
        fichier.write(fichier_cnf)
    return fichier_cnf

In [13]:
# test
e=encoder(3,4, 'test.cnf')
print(e)

p cnf 36 222
-1 -4 0
-1 -7 0
-1 -1 0
-1 -2 0
-1 -3 0
-4 -7 0
-4 -1 0
-4 -2 0
-4 -3 0
-7 -1 0
-7 -2 0
-7 -3 0
-1 -2 0
-1 -3 0
-2 -3 0
-2 -5 0
-2 -8 0
-2 -4 0
-2 -5 0
-2 -6 0
-5 -8 0
-5 -4 0
-5 -5 0
-5 -6 0
-8 -4 0
-8 -5 0
-8 -6 0
-4 -5 0
-4 -6 0
-5 -6 0
-3 -6 0
-3 -9 0
-3 -7 0
-3 -8 0
-3 -9 0
-6 -9 0
-6 -7 0
-6 -8 0
-6 -9 0
-9 -7 0
-9 -8 0
-9 -9 0
-7 -8 0
-7 -9 0
-8 -9 0
-10 -13 0
-10 -16 0
-10 -10 0
-10 -11 0
-10 -12 0
-13 -16 0
-13 -10 0
-13 -11 0
-13 -12 0
-16 -10 0
-16 -11 0
-16 -12 0
-10 -11 0
-10 -12 0
-11 -12 0
-11 -14 0
-11 -17 0
-11 -13 0
-11 -14 0
-11 -15 0
-14 -17 0
-14 -13 0
-14 -14 0
-14 -15 0
-17 -13 0
-17 -14 0
-17 -15 0
-13 -14 0
-13 -15 0
-14 -15 0
-12 -15 0
-12 -18 0
-12 -16 0
-12 -17 0
-12 -18 0
-15 -18 0
-15 -16 0
-15 -17 0
-15 -18 0
-18 -16 0
-18 -17 0
-18 -18 0
-16 -17 0
-16 -18 0
-17 -18 0
-19 -22 0
-19 -25 0
-19 -19 0
-19 -20 0
-19 -21 0
-22 -25 0
-22 -19 0
-22 -20 0
-22 -21 0
-25 -19 0
-25 -20 0
-25 -21 0
-19 -20 0
-19 -21 0
-20 -21 0
-20 -23 0
-20 -26 0
-20 -22

In [14]:
nb_clauses = len(e.split("0\n"))-1
print(f'nombre de clauses au total : {nb_clauses}')
nb_variables = nj*(ne**2)
print(f'nombre de variables au total : {nb_variables}')

nombre de clauses au total : 222
nombre de variables au total : 36


# Question 3

Le fichier est enregistré dans le dossier simp
On lance la commande: glucose_static -model SAT_36_222.cnf
Le solveur indique que le problème est insatisfiable. Cette réponse était attendue en effet pour satisfaire les contraintes il aurait fallu au moins 6 jours.

# Question 4

Ecrire une fonction ou un programme decoder qui prend pour argument un fichier contenant la sortie de l’appel à glucose (plus éventuellement nj et ne) et qui traduit le modèle rendu en une solution du problème de planning des matchs affichée lisiblement. On peut faire appel a un fichier extérieur donnant le nom des équipes : une par ligne dans leur ordre de numérotation.

Tout d'abord rediriger la sortie en ligne de commande :<br>
glucose_static -model SAT_nbvar_nbclauses.cnf new_file

In [15]:
def decoder(ne, nj, entre, affiche=True,equipe = None ):
    with  open(entre, 'r') as fp:
        resultat=fp.readline()
        if resultat=='UNSAT\n':
            if affiche :
                print("Insatisfiable")
            return False
        
        resultat=resultat.split(' ')
        code = [int(k) for k in resultat]
    
    if affiche:
        jour = -1
        for k in code: 
            if k > 0:
                j,x,y = decodage(k,ne)
                if j != jour:
                    jour = j
                    if j%2 == 0 or j == 0:
                        print(f"Mercredi Semaine {j//2} :")
                    else:                      
                        print(f"Dimanche Semaine {j//2} :")
                
                if equipe == None or equipe == [] or len(equipe) != ne:
                    print(f"\tEquipe {x} contre Equipe {y}")
                else:
                    print(f"\t{equipe[x]} contre {equipe[y]}")
                
    return True

# Question 5

In [17]:
def planning(ne, nj,equipe=None, affiche=True, timeout=None) :
    ##encodage
    nv=nj*ne**2 #nombre de var
    nc=ne*(ne-1) * (nj*(nj-1)//2+1)  +  ne*nj * 2*ne*(2*ne-1)//2 #nombre de contraintes
    encoder(ne,nj,'systeme.cnf')
    
    ##execution du solveur
    try : 
        subprocess.run("./glucose_static systeme.cnf sortie",timeout=timeout,shell=True,stdout=subprocess.DEVNULL)
        return decoder (ne, nj, 'sortie',affiche=affiche, equipe=equipe )
    except : 
        if affiche :
            print("TimeOut")
        return None

In [18]:
equipe = ['Real_madrid', 'PSG', 'Bayern_munich',"Manchester_united"]
planning(4,6,equipe, affiche=True,timeout=20)

Mercredi Semaine 0 :
	Bayern_munich contre PSG
	Manchester_united contre Real_madrid
Dimanche Semaine 0 :
	PSG contre Real_madrid
	Manchester_united contre Bayern_munich
Mercredi Semaine 1 :
	Real_madrid contre PSG
	Bayern_munich contre Manchester_united
Dimanche Semaine 1 :
	Real_madrid contre Manchester_united
	PSG contre Bayern_munich
Mercredi Semaine 2 :
	Real_madrid contre Bayern_munich
	PSG contre Manchester_united
Dimanche Semaine 2 :
	Bayern_munich contre Real_madrid
	Manchester_united contre PSG


True

In [19]:
with open('./ligue_1.txt','r') as f:
    equipe = [i[:-1] for i in f.readlines()]
print(equipe)

['AC Ajaccio', 'Angers', 'Auxerre', 'Brest', 'Clermont', 'Lens', 'Lille', 'Lorient', 'Lyon', 'Marseille', 'Monaco', 'Montpellier', 'Nantes', 'Nice', 'Paris SG', 'Reims', 'Rennes', 'Strasbourg', 'Toulouse', 'Troyes']


In [20]:
planning(20,40,equipe,affiche=True,timeout=10)

Mercredi Semaine 0 :
	Angers contre Brest
	Clermont contre Monaco
	Lens contre Nantes
	Lorient contre Montpellier
	Lyon contre Lille
	Marseille contre Paris SG
	Reims contre AC Ajaccio
	Strasbourg contre Rennes
	Troyes contre Toulouse
Dimanche Semaine 0 :
	Angers contre Monaco
	Auxerre contre Troyes
	Brest contre Toulouse
	Lens contre Strasbourg
	Lorient contre Clermont
	Marseille contre Lyon
	Montpellier contre AC Ajaccio
	Nice contre Rennes
	Paris SG contre Reims
Mercredi Semaine 1 :
	Brest contre Lyon
	Clermont contre Marseille
	Lens contre Angers
	Montpellier contre Nantes
	Nice contre AC Ajaccio
	Paris SG contre Toulouse
	Reims contre Lorient
	Rennes contre Auxerre
	Strasbourg contre Lille
	Troyes contre Monaco
Dimanche Semaine 1 :
	AC Ajaccio contre Toulouse
	Angers contre Lens
	Clermont contre Troyes
	Lille contre Auxerre
	Lyon contre Lorient
	Marseille contre Rennes
	Monaco contre Montpellier
	Nantes contre Nice
	Reims contre Brest
	Strasbourg contre Paris SG
Mercredi Semaine 2

True

# Exercice 4

In [25]:
def borne_min(ne,timeout=10) :
    nj=0
    plan = False
    while plan == False :
        nj+=1
        plan = planning(ne, nj, equipe =None,affiche=False, timeout=timeout)
        if plan == None :
            return None
    return nj

In [28]:
for ne in range(3,11):
    nj = borne_min(ne,10)
    print(f'ne:{ne},nj:{nj}')

ne:3,nj:6
ne:4,nj:6
ne:5,nj:None
ne:6,nj:None
ne:7,nj:None
ne:8,nj:None
ne:9,nj:None
ne:10,nj:None
